# Random Forests, GBM, XGBoost, LightGBM, CatBoost


In [51]:
!pip install catboost


In [52]:
!pip install xgboost


In [53]:
!pip install lightgbm

In [54]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV, cross_validate, RandomizedSearchCV, validation_curve

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [55]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

warnings.simplefilter(action='ignore', category=Warning)

In [56]:
df = pd.read_csv("C:/Users/esman/PycharmProjects/feature_engineering/datasets/diabetes.csv")


In [57]:
y = df["Outcome"]
X = df.drop(["Outcome"], axis=1)

# 1.RANDOM FOREST UYGULAMA

In [58]:
#ilk başda işlemlere başlamadan önce bir hatama bakmak istiyorum.
#modelleme öncesindeki yani hiperparametre optimizasyonu yapmadan önceki değerler.
cv_results = cross_validate(rf_model, X, y, cv=10, scoring=["accuracy", "f1", "roc_auc"])


In [59]:
cv_results['test_accuracy'].mean()


0.766848940533151

In [60]:
cv_results['test_f1'].mean()


0.6447777811143756

In [61]:
cv_results['test_roc_auc'].mean()


0.8271054131054132

In [62]:
rf_model = RandomForestClassifier(random_state=17)
rf_model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 17,
 'verbose': 0,
 'warm_start': False}

In [63]:
#dikkat ön tanımlı argümanlarınıda giriyorum.
rf_params = {"max_depth": [5, 8, None],
             "max_features": [3, 5, 7, "auto"],
             "min_samples_split": [2, 5, 8, 15, 20],
             "n_estimators": [100, 200, 500]}

In [64]:
#gridsearchCV methoduyla bu parametrelerde arama yapıyorum.
rf_best_grid = GridSearchCV(rf_model, rf_params, cv=5, n_jobs=-1, verbose=True).fit(X, y)


Fitting 5 folds for each of 180 candidates, totalling 900 fits


In [65]:
#en iyi parametreleri elde etmek için:
rf_best_grid.best_params_

{'max_depth': None,
 'max_features': 5,
 'min_samples_split': 8,
 'n_estimators': 500}

In [66]:
#final model oluştur.
rf_final = rf_model.set_params(**rf_best_grid.best_params_, random_state=17).fit(X, y)

In [67]:

cv_results = cross_validate(rf_final, X, y, cv=10, scoring=["accuracy", "f1", "roc_auc"])


In [68]:
cv_results['test_accuracy'].mean()


0.766848940533151

In [69]:
cv_results['test_f1'].mean()


0.6447777811143756

In [70]:
cv_results['test_roc_auc'].mean()

0.8271054131054132

In [1]:
#yukarıdaki parametre değerleriyle, çalışmanın başında baktığın parametre değerlerini karşılaştır.


In [2]:
#feature importance
def plot_importance(model, features, num=len(X), save=False):
    feature_imp = pd.DataFrame({'Value': model.feature_importances_, 'Feature': features.columns})
    plt.figure(figsize=(10, 10))
    sns.set(font_scale=1)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value",
                                                                     ascending=False)[0:num])
    plt.title('Features')
    plt.tight_layout()
    plt.show()
    if save:
        plt.savefig('importances.png')
        
#bütün değişkenlerin önem sırasına göre grafik çıkarır. 
#eğer mesela en önemli 5 değişken istiyorsam; plot_importance(rf_final, X, num=5 )
plot_importance(rf_final, X)

NameError: name 'X' is not defined

In [ ]:
#validation curve çalıştır.
def val_curve_params(model, X, y, param_name, param_range, scoring="roc_auc", cv=10):
    train_score, test_score = validation_curve(
        model, X=X, y=y, param_name=param_name, param_range=param_range, scoring=scoring, cv=cv)

    mean_train_score = np.mean(train_score, axis=1)
    mean_test_score = np.mean(test_score, axis=1)

    plt.plot(param_range, mean_train_score,
             label="Training Score", color='b')

    plt.plot(param_range, mean_test_score,
             label="Validation Score", color='g')

    plt.title(f"Validation Curve for {type(model).__name__}")
    plt.xlabel(f"Number of {param_name}")
    plt.ylabel(f"{scoring}")
    plt.tight_layout()
    plt.legend(loc='best')
    plt.show(block=True)

#max_Depth göre bir değerlendirme yapalım.
val_curve_params(rf_final, X, y, "max_depth", range(1, 11), scoring="roc_auc")

# 2. Gradient Boosting Machines ( GBM )
Artık optimizasyon sürecine dayalı olarak çalışan bir ağaç yöntemidir.
GBM' ,in temelleri AdaBoost atmaktadır.

In [71]:
#modellemeye başlamadan önce yine bir hiperparametre optimizasyonundan önceki hatalara bakalım.
cv_results = cross_validate(gbm_model, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])
cv_results['test_accuracy'].mean()
# 0.7591715474068416
cv_results['test_f1'].mean()
# 0.634
cv_results['test_roc_auc'].mean()
# 0.82548

NameError: name 'gbm_model' is not defined

In [72]:
#modelimizi tanımlayalım
gbm_model = GradientBoostingClassifier(random_state=17)

In [73]:
#ön tanımlı değerlerine bakalım.
#n_estimaators=optimizasyon sayısıdır. kaç defa boost ettin.
gbm_model.get_params()


{'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'log_loss',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': 17,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [74]:
#ön tanımlı değerleri barındaracak şekilde değerlerle oynayabilirisn.
#leatning rate oranını ne kadar düşük girersen o kadar başarılı sonuçlar alırısn.
#subsample= göz önünde bulundurulacak gözlem oranı
gbm_params = {"learning_rate": [0.01, 0.1],
              "max_depth": [3, 8, 10],
              "n_estimators": [100, 500, 1000],
              "subsample": [1, 0.5, 0.7]}


In [75]:
#en iyi parametre değerleri
gbm_best_grid = GridSearchCV(gbm_model, gbm_params, cv=5, n_jobs=-1, verbose=True).fit(X, y)


Fitting 5 folds for each of 54 candidates, totalling 270 fits


In [76]:
gbm_best_grid.best_params_


{'learning_rate': 0.1, 'max_depth': 8, 'n_estimators': 100, 'subsample': 0.5}

In [77]:
#final modelini kur
gbm_final = gbm_model.set_params(**gbm_best_grid.best_params_, random_state=17, ).fit(X, y)


In [ ]:
#final modelinde ki başarı için metriklerini hesapla. ve önceki değerlerle karşılaştır.
cv_results = cross_validate(gbm_model, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])

In [ ]:
cv_results['test_accuracy'].mean()


In [ ]:
cv_results['test_f1'].mean()


In [ ]:
cv_results['test_roc_auc'].mean()


# 3.XGBoost

In [ ]:
cv_results = cross_validate(xgboost_model, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])
cv_results['test_accuracy'].mean()
# 0.75265
cv_results['test_f1'].mean()
# 0.631
cv_results['test_roc_auc'].mean()
# 0.7987

In [ ]:
xgboost_model = XGBClassifier(random_state=17, use_label_encoder=False)
xgboost_model.get_params()

In [ ]:
xgboost_params = {"learning_rate": [0.1, 0.01],
                  "max_depth": [5, 8],
                  "n_estimators": [100, 500, 1000],
                  "colsample_bytree": [0.7, 1]}  #colsample ile subsample aynı şey. dokümantasyondan incele.

In [ ]:
xgboost_best_grid = GridSearchCV(xgboost_model, xgboost_params, cv=5, n_jobs=-1, verbose=True).fit(X, y)


In [ ]:
#final model kuralım
xgboost_final = xgboost_model.set_params(**xgboost_best_grid.best_params_, random_state=17).fit(X, y)


In [ ]:
#model başarısı için parametre değerlerine bak.
cv_results = cross_validate(xgboost_final, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])


In [ ]:
cv_results['test_accuracy'].mean()


In [ ]:
cv_results['test_f1'].mean()


In [ ]:
cv_results['test_roc_auc'].mean()


# 4.LİGHTGBM
hızlıdır.
- teorikte randomforest, uygulamada lightgbm tercih etmek daha doğru olabilir.
- LightGBM derinlemesine bir arama yaptığı için değişken öenm düzeyleri değişebilir. bu nednele eğer özellik seçimi (feature importance) işlemleri yapacaksam; 
randomforest ve xgboost bakarak karar vermeye çalış.

LİGHTGBM in en önemli parametresi; n_estimators dır. tahmin sayısı, işlem sayısı (iterasyon sayısı), boostring sayısıdır.
lightgbm de 5000,10000 gibi n_estimators değerleirni mutlaka denemeliyiz.

In [ ]:
#Modeli kurmadan önce parametre değerleirne bak.
cv_results = cross_validate(xgboost_model, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])
cv_results['test_accuracy'].mean()
# 0.75265
cv_results['test_f1'].mean()
# 0.631
cv_results['test_roc_auc'].mean()
# 0.7987

In [ ]:
#modelleme
lgbm_model = LGBMClassifier(random_state=17)
lgbm_model.get_params()


In [ ]:
#parametre setini oluştur.
lgbm_params = {"learning_rate": [0.01, 0.1],
               "n_estimators": [100, 300, 500, 1000],
               "colsample_bytree": [0.5, 0.7, 1]}

In [ ]:
lgbm_best_grid = GridSearchCV(lgbm_model, lgbm_params, cv=5, n_jobs=-1, verbose=True).fit(X, y)

In [ ]:
lgbm_final = lgbm_model.set_params(**lgbm_best_grid.best_params_, random_state=17).fit(X, y)

In [ ]:
#şimdiki metriklerin değerleri daha yüksek o zaman dahha başarılıyım 
cv_results = cross_validate(lgbm_final, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])

cv_results['test_accuracy'].mean()
cv_results['test_f1'].mean()
cv_results['test_roc_auc'].mean()

In [ ]:
# Hiperparametre yeni değerlerle
lgbm_params = {"learning_rate": [0.01, 0.02, 0.05, 0.1],
               "n_estimators": [200, 300, 350, 400],
               "colsample_bytree": [0.9, 0.8, 1]}

lgbm_best_grid = GridSearchCV(lgbm_model, lgbm_params, cv=5, n_jobs=-1, verbose=True).fit(X, y)

lgbm_final = lgbm_model.set_params(**lgbm_best_grid.best_params_, random_state=17).fit(X, y)

cv_results = cross_validate(lgbm_final, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])

cv_results['test_accuracy'].mean()
cv_results['test_f1'].mean()
cv_results['test_roc_auc'].mean()

In [ ]:
# Hiperparametre optimizasyonu sadece n_estimators için.
lgbm_model = LGBMClassifier(random_state=17, colsample_bytree=0.9, learning_rate=0.01)

lgbm_params = {"n_estimators": [200, 400, 1000, 5000, 8000, 9000, 10000]}

lgbm_best_grid = GridSearchCV(lgbm_model, lgbm_params, cv=5, n_jobs=-1, verbose=True).fit(X, y)

lgbm_final = lgbm_model.set_params(**lgbm_best_grid.best_params_, random_state=17).fit(X, y)

cv_results = cross_validate(lgbm_final, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])

cv_results['test_accuracy'].mean()
cv_results['test_f1'].mean()
cv_results['test_roc_auc'].mean()

In [ ]:
#şimdi en iyi parametre değerine bak kaç olmuş?
lgbm_best_grid.best_params_

# 5.  CATBOOST

In [ ]:
catboost_model = CatBoostClassifier(random_state=17, verbose=False)


In [ ]:
#hiperparametre optimizasyonu yapacağım. yapmadan önce başarı metriklerime bakalım.
cv_results = cross_validate(catboost_model, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])

cv_results['test_accuracy'].mean()
cv_results['test_f1'].mean()
cv_results['test_roc_auc'].mean()

In [ ]:
#hiperparametre optimizyanu
catboost_params = {"iterations": [200, 500],
                   "learning_rate": [0.01, 0.1],
                   "depth": [3, 6]}

In [ ]:
catboost_best_grid = GridSearchCV(catboost_model, catboost_params, cv=5, n_jobs=-1, verbose=True).fit(X, y)



In [ ]:
catboost_final = catboost_model.set_params(**catboost_best_grid.best_params_, random_state=17).fit(X, y)

In [ ]:
cv_results = cross_validate(catboost_final, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])

cv_results['test_accuracy'].mean()
cv_results['test_f1'].mean()
cv_results['test_roc_auc'].mean()

# MODELLERİ FUTURE İMPORTANCE BAZINDA İNCELEMEK
importance skoru 0 a yakın olan değişkenlerle ilgilenmem. çünkü bunlar hiperparametre optimizasyonu sürecini uzatır. hazırlanacak olan dokümantasyonları uzatır.
gereksiz future türetmekde mantıksız. gerçekten modelin başarısını olumlu etkiliyorsa bunları kullanmalısın.
Mkine öğrenimi model geliştirme sürecindetemel amacımız; "mümkün olan en basit haliyle mümkün olan en başarılı modeli kullanmaktır."

biz future seçmek istediğimizde, farklı modellerin future importance düzeylerine bakıp karar verebilriz. 

In [ ]:

def plot_importance(model, features, num=len(X), save=False):
    feature_imp = pd.DataFrame({'Value': model.feature_importances_, 'Feature': features.columns})
    plt.figure(figsize=(10, 10))
    sns.set(font_scale=1)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value",
                                                                     ascending=False)[0:num])
    plt.title('Features')
    plt.tight_layout()
    plt.show()
    if save:
        plt.savefig('importances.png')

In [ ]:
#randomforest da future importance

plot_importance(rf_final, X)

In [ ]:
#gbm de future importance

plot_importance(gbm_final, X)

In [ ]:
#xgboost da future importance

plot_importance(xgboost_final, X)


In [ ]:
#lightgbm de future importance

plot_importance(lgbm_final, X)


In [ ]:
#catboost da future importance

plot_importance(catboost_final, X)

# HİPERPARAMETRE OPTİMİZASYONU İÇİN RANDOMSEARCHCV
gridsearchcv --> verilen bir setin olası bütün kombinasyonlarını tek tek dener. randome göre daha uzun sürer.

randomsearchcv --> verilecek bir hiperparametre seti içeriisnden rastgele seçimler yapar. ve bu rastgele seçimleri arar. 

In [ ]:
rf_model = RandomForestClassifier(random_state=17)


In [ ]:
rf_random_params = {"max_depth": np.random.randint(5, 50, 10),   #5 den 50 ye kadar 10 tane sayı oluştur.
                    "max_features": [3, 5, 7, "auto", "sqrt"],
                    "min_samples_split": np.random.randint(2, 50, 20), # 2 den 50 ye kadar 20 tane sayı oluştur.
                    "n_estimators": [int(x) for x in np.linspace(start=200, stop=1500, num=10)]}  #200 den 1500 e kadar 10 tane sayı oluştur. 

In [ ]:
rf_random = RandomizedSearchCV(estimator=rf_model,
                               param_distributions=rf_random_params,
                               n_iter=100,  # denenecek parametre sayısı
                               cv=3,
                               verbose=True,
                               random_state=42,
                               n_jobs=-1)

rf_random.fit(X, y)

In [ ]:
rf_random.best_params_


In [ ]:
rf_random_final = rf_model.set_params(**rf_random.best_params_, random_state=17).fit(X, y)


In [ ]:
cv_results = cross_validate(rf_random_final, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])

cv_results['test_accuracy'].mean()
cv_results['test_f1'].mean()
cv_results['test_roc_auc'].mean()


# ÖĞRENME EĞRİLERİYLE MODEL KARMAŞIKLIĞINI İNCELEME

In [ ]:
def val_curve_params(model, X, y, param_name, param_range, scoring="roc_auc", cv=10):
    train_score, test_score = validation_curve(
        model, X=X, y=y, param_name=param_name, param_range=param_range, scoring=scoring, cv=cv)

    mean_train_score = np.mean(train_score, axis=1)
    mean_test_score = np.mean(test_score, axis=1)

    plt.plot(param_range, mean_train_score,
             label="Training Score", color='b')

    plt.plot(param_range, mean_test_score,
             label="Validation Score", color='g')

    plt.title(f"Validation Curve for {type(model).__name__}")
    plt.xlabel(f"Number of {param_name}")
    plt.ylabel(f"{scoring}")
    plt.tight_layout()
    plt.legend(loc='best')
    plt.show(block=True)

In [ ]:
rf_val_params = [["max_depth", [5, 8, 15, 20, 30, None]],
                 ["max_features", [3, 5, 7, "auto"]],
                 ["min_samples_split", [2, 5, 8, 15, 20]],
                 ["n_estimators", [10, 50, 100, 200, 500]]]

In [ ]:
rf_model = RandomForestClassifier(random_state=17)


In [ ]:
#ilgili parametrelerdeki farkklılıklar net bir şekilde görülür.
for i in range(len(rf_val_params)):
    val_curve_params(rf_model, X, y, rf_val_params[i][0], rf_val_params[i][1])



In [ ]:
rf_val_params[0][1]